In [1]:
from glob import glob
import os.path as op
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline
import nibabel as nib
from nibabel.streamlines import load as load_trk
import dipy.tracking.streamline as dts
import dipy.tracking.utils as dtu
from skimage.transform import resize
from scipy.ndimage.morphology import binary_dilation
import dipy.data as dpd
from sklearn.utils import class_weight
from sklearn.cross_validation import train_test_split

import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model

import bundlenet as bn

import dask.bag as db
import warnings
warnings.filterwarnings('ignore')

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
num_classes=16
dropout_factor=0
num_convlayers=3
num_fulllayers=1
model = load_model('savedmodels/savedmodel_nclasses'+ str(num_classes) + '_do' + str(dropout_factor) + '_nc' + str(num_convlayers) + '_fc' + str(num_fulllayers) +'.h5')

In [3]:
sub2_t1_img = nib.load('/home/ubuntu/MNI152_T1_1mm_brain.nii.gz')
vol_shape_subj2=sub2_t1_img.shape[0:3]
size_slimage = 100
dil_iters = 5

In [ ]:
num_classes = 16
for subj in '970764','970764','971160','972566','973770','978578','979984','983773','984472':
    if op.exists('./streamlines_processed/streamlines_processed_' + subj + '_' + str(num_classes) + '.npz'):
        print(subj)
        #loaded_from_file = np.load('streamlines_processed_' + subj + '_' + str(num_classes) + '.npz')
        #streamlines_all_processed_subj2 = loaded_from_file['processed_streamlines']
        #labels_selected_s2 = loaded_from_file['streamline_labels']
    else:
        streamlines_all_s2 = bn.read_sl_mni('/home/ubuntu/copied16/sub-' + subj + '/sess-01/dwi/out_work/whole_brain_MNI_tracks_from_peaks__moved.trk') 
        map_files = sorted(glob('/home/ubuntu/copied16/sub-' + subj + '/sess-01/dwi/out_work/rrbs/whole_brain_MNI_tracks_from_peaks__moved_*.npy'))
        labeled_index_s2, labels_s2 = bn.getlabeledstreamlines(map_files)
        #unlabeled_index_s2 = bn.getunlabeledstreamlines(len(streamlines_all_s2), labeled_index_s2, len(streamlines_all_s2)-len(labeled_index_s2), 0)
        #labels_selected_s2, streamlines_selected_s2 = bn.combinestreamlines(labeled_index_s2, unlabeled_index_s2, labels_s2, streamlines_all_s2)
        labels = 16*np.ones(len(streamlines_all_s2))
        ii=0
        for i in labeled_index_s2:
            labels[int(i)]=labels_s2[ii]
            ii=ii+1
        streamlines_all_processed_subj2 = bn.process_sl(streamlines_all_s2,-1,vol_shape_subj2,size_slimage,dil_iters)
        np.savez('streamlines_processed/streamlines_processed_' + subj + '_' + str(num_classes) + '.npz',processed_streamlines = streamlines_all_processed_subj2, streamline_labels = labels)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
159000
160000
161000
162000
163000
164000
165000

In [33]:
labeled_index_s2, labels_s2 = bn.getlabeledstreamlines(map_files)
labels_s2
labels = 16*np.ones(len(streamlines_all_s2))
ii=0
for i in labeled_index_s2:
    labels[int(i)]=labels_s2[ii]
    ii=ii+1

In [ ]:
num_classes = 42
for subj in '970764','970764','971160','972566','973770','978578','979984','983773','984472':
    if op.exists('./streamlines_processed/streamlines_processed' + subj + '_' + str(num_classes) + '.npz'):
        print(subj)
        #loaded_from_file = np.load('streamlines_processed' + subj + '_' + str(num_classes) + '.npz')
        #streamlines_all_processed_subj1 = loaded_from_file['processed_streamlines']
        #labels_selected_s1 = loaded_from_file['streamline_labels']
    else:
        print(subj)
        streamlines_all_s1 = bn.read_sl_mni('/home/ubuntu/copied80/sub-'+ subj + '/sess-01/dwi/out_work/whole_brain_MNI_tracks_from_peaks__moved.trk') 
        map_files = []
        for b in bundles80:
            map_files.append('/home/ubuntu/copied80/sub-'+ subj + '/sess-01/dwi/out_work/rrbs/whole_brain_MNI_tracks_from_peaks__moved_' + b + '_R__labels.npy')
        for b in bundles80:
            map_files.append('/home/ubuntu/copied80/sub-'+ subj + '/sess-01/dwi/out_work/rrbs/whole_brain_MNI_tracks_from_peaks__moved_' + b + '_L__labels.npy')
        map_files.append('/home/ubuntu/copied80/sub-'+ subj + '/sess-01/dwi/out_work/rrbs/whole_brain_MNI_tracks_from_peaks__moved_F_L_R__labels.npy')
        map_files.append('/home/ubuntu/copied80/sub-'+ subj + '/sess-01/dwi/out_work/rrbs/whole_brain_MNI_tracks_from_peaks__moved_CC_ForcepsMajor__labels.npy')
        map_files.append('/home/ubuntu/copied80/sub-'+ subj + '/sess-01/dwi/out_work/rrbs/whole_brain_MNI_tracks_from_peaks__moved_CC_ForcepsMinor__labels.npy')
        map_files.append('/home/ubuntu/copied80/sub-'+ subj + '/sess-01/dwi/out_work/rrbs/whole_brain_MNI_tracks_from_peaks__moved_CCMid__labels.npy')
        map_files = sorted(map_files)    
        labeled_index_s1, labels_s1 = bn.getlabeledstreamlines(map_files)
        unlabeled_index_s1 = bn.getunlabeledstreamlines(len(streamlines_all_s1), labeled_index_s1, len(streamlines_all_s1)-len(labeled_index_s1), 0)
        labels_selected_s1, streamlines_selected_s1 = bn.combinestreamlines(labeled_index_s1, unlabeled_index_s1, labels_s1, streamlines_all_s1)
        streamlines_all_processed_subj1 = bn.process_sl(streamlines_selected_s1,-1,vol_shape_subj2,size_slimage,dil_iters)
        np.savez('streamlines_processed/streamlines_processed' + subj + '_' + str(num_classes) + '.npz',processed_streamlines = streamlines_all_processed_subj1, streamline_labels = labels_selected_s1)